In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os
# Set paths
biomed_path = "/content/drive/MyDrive/BIOMED"
ham10000_path = os.path.join(biomed_path, "HAM10000")

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers



# Load metadata
metadata = pd.read_csv(os.path.join(biomed_path, "HAM10000_metadata.csv"))

# Update the image_id column to include the full path to the images
metadata["image_id"] = metadata.apply(lambda row: os.path.join(biomed_path, "HAM10000_images_part_1", row["image_id"] + ".jpg") if row["image_id"][0] == 'I' else os.path.join(biomed_path, "HAM10000_images_part_2", row["image_id"] + ".jpg"), axis=1)

# Split the dataset into training, validation, and test sets
train_df, test_df = train_test_split(metadata, test_size=0.2, random_state=42, stratify=metadata["dx"])
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42, stratify=train_df["dx"])

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

image_size = 240
batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col="image_id",
    y_col="dx",
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical",
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col="image_id",
    y_col="dx",
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical",
)

# Load EfficientNetB0 model
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(7, activation="softmax", kernel_regularizer=regularizers.l2(0.01))(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
num_epochs = 25
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.n // batch_size,
    epochs=num_epochs,
)

# Save the trained model
model.save("efficientnetb0_ham10000.h5")


Found 3042 validated image filenames belonging to 7 classes.


/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 2967 invalid image filename(s) in x_col="image_id". These filename(s) will be ignored.
  warnings.warn(


Found 968 validated image filenames belonging to 7 classes.


/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 1035 invalid image filename(s) in x_col="image_id". These filename(s) will be ignored.
  warnings.warn(


16705208/16705208 [==============================] - 0s 0us/step


Epoch 1/25
95/95 [==============================] - 1472s 15s/step - loss: 0.8310 - accuracy: 0.7518 - val_loss: 1.9016 - val_accuracy: 0.7312
Epoch 2/25
95/95 [==============================] - 1245s 13s/step - loss: 0.6050 - accuracy: 0.8116 - val_loss: 0.7222 - val_accuracy: 0.7812
Epoch 3/25
95/95 [==============================] - 1274s 13s/step - loss: 0.5074 - accuracy: 0.8342 - val_loss: 0.7201 - val_accuracy: 0.7875
Epoch 4/25
95/95 [==============================] - 1286s 14s/step - loss: 0.4528 - accuracy: 0.8530 - val_loss: 1.1079 - val_accuracy: 0.6958
Epoch 5/25
52/95 [===============>..............] - ETA: 8:31 - loss: 0.3919 - accuracy: 0.8672